In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
gis = arcgis.GIS()

from IPython.display import display, Markdown

In [2]:
#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_data     = os.path.join(dir_working, "data"        )
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "inputs"      )
dir_webapp   = os.path.join(dir_working, "forWebApp"   )

ClassParameters    = os.path.join(dir_inputs, r"class_parameters_scenario1.csv"     )

ProcessGDB = "process.gdb"
WebAppGDB  = "centerstod_webapp.gdb"
ParcelsDevCategory = os.path.join(dir_process, ProcessGDB + "\ParcelsAA_DevCategory")
ParcelsWebApp      = os.path.join(dir_webapp , WebAppGDB  + "\AAParcels"            )
DevCatWebApp       = os.path.join(dir_webapp , WebAppGDB  + "\AADevCategories"      )

#name of new data features
AnalysisAreas      = os.path.join(dir_process, "analysisareas.shp"        )
AnalysisAreas_IDs  = os.path.join(dir_process, "analysisareas_ids.shp"    )
ParcelsAA          = os.path.join(dir_process, "parcelsaa.shp"            )
AnalysisAreasTable = os.path.join(dir_results, "areas.csv"                )

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)


e:\GitHub\Centers-Capacity-Tool-byCity


In [3]:
AnalysisAreas

'e:\\GitHub\\Centers-Capacity-Tool-byCity\\intermediate\\analysisareas.shp'

In [4]:
sdf_AA = pd.DataFrame.spatial.from_featureclass(AnalysisAreas)
sdf_AA

,FID,FID_Munici,COUNTYNBR,NAME,COUNTYSEAT,SHORTDESC,UPDATED,FIPS,ENTITYNBR,SALESTAXID,...,FID_TODCen,AreaName,AreaType,Area,ClassName,Shape_Le_1,Shape_Area,AnalysisAr,Analysis_1,SHAPE
0,0,5,18,Bluffdale,0,BLUFFDALE (SL CO),2018-11-15,06810,3020.0,019,...,64,Bluffdale - Downtown Riverton,City Center,WFRC MPO,NONTOD|City Center,0.002517,1.025230e-07,Bluffdale - Downtown Riverton,NONTOD|City Center,"{""rings"": [[[-111.93828708318233, 40.500645841..."
1,1,5,18,Bluffdale,0,BLUFFDALE (SL CO),2018-11-15,06810,3020.0,019,...,77,Bluffdale - Herriman Porter Rockwell Center,City Center,WFRC MPO,NONTOD|City Center,0.014422,1.756575e-06,Bluffdale - Herriman Porter Rockwell Center,NONTOD|City Center,"{""rings"": [[[-111.94214447958404, 40.460508711..."
2,2,5,18,Bluffdale,0,BLUFFDALE (SL CO),2018-11-15,06810,3020.0,019,...,82,Bluffdale - Independence,City Center,WFRC MPO,NONTOD|City Center,0.081911,3.457529e-04,Bluffdale - Independence,NONTOD|City Center,"{""rings"": [[[-111.90309291350712, 40.486177585..."
3,3,5,18,Bluffdale,0,BLUFFDALE (SL CO),2018-11-15,06810,3020.0,019,...,121,Bluffdale - Prison Site,Urban Center,WFRC MPO,NONTOD|Urban Center,0.060180,6.782125e-05,Bluffdale - Prison Site,NONTOD|Urban Center,"{""rings"": [[[-111.90309291350712, 40.486177585..."
4,4,7,18,Herriman,0,HERRIMAN TOWN,2019-11-22,34970,3035.0,060,...,76,Herriman - Herriman Olympia Hills,City Center,WFRC MPO,NONTOD|City Center,0.032693,6.718727e-07,Herriman - Herriman Olympia Hills,NONTOD|City Center,"{""rings"": [[[-112.05317660771425, 40.526060430..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,289,169,18,West Valley City,0,WEST VALLEY CITY,2012-02-02,83470,3120.0,167,...,191,West Valley City - Fairbourne Station,Urban Center,WFRC MPO,LRT|Urban Center,0.028093,4.651836e-05,West Valley City - Fairbourne Station,LRT|Urban Center,"{""rings"": [[[-111.95988335513272, 40.697973328..."
290,290,223,18,Salt Lake City,1,SALT LAKE CITY,2018-11-14,67000,3070.0,122,...,46,Salt Lake City - 1940 W NORTH TEMPLE LRT NC,,WFRC MPO,LRT|NA,0.020945,2.054510e-05,Salt Lake City - 1940 W NORTH TEMPLE LRT NC,LRT|NA,"{""rings"": [[[-111.94851766915508, 40.773298628..."
291,291,223,18,Salt Lake City,1,SALT LAKE CITY,2018-11-14,67000,3070.0,122,...,205,Salt Lake City - North Temple Center,City Center,WFRC MPO,LRT|City Center,0.020945,2.054510e-05,Salt Lake City - North Temple Center,LRT|City Center,"{""rings"": [[[-111.94851766915508, 40.773298628..."
292,292,226,29,Roy,0,ROY CITY,2018-11-09,65110,3090.0,037,...,208,Roy - Midland Drive - 3500 West,City Center,WFRC MPO,NONTOD|City Center,0.002902,2.813621e-11,Roy - Midland Drive - 3500 West,NONTOD|City Center,"{""rings"": [[[-112.06303223395042, 41.164103772..."


In [5]:
#create map centered on Salt Lake
map_areas = gis.map('Salt Lake')

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[128,128,128,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#classify land use
arcade_expression_aa = ("var v = $feature.Analysis_1;"
                        "if      (v=='NONTOD|Metropolitan Center') { return 'mc'    ; }"
                        "else if (v=='CRT|Metropolitan Center'   ) { return 'mc_tod'; }"
                        "else if (v=='LRT|Metropolitan Center'   ) { return 'mc_tod'; }"
                        "else if (v=='NONTOD|Urban Center'       ) { return 'uc'    ; }"
                        "else if (v=='CRT|Urban Center'          ) { return 'uc_tod'; }"
                        "else if (v=='LRT|Urban Center'          ) { return 'uc_tod'; }"
                        "else if (v=='NONTOD|City Center'        ) { return 'cc'    ; }"
                        "else if (v=='CRT|City Center'           ) { return 'cc_tod'; }"
                        "else if (v=='LRT|City Center'           ) { return 'cc_tod'; }"
                        "else if (v=='CRT|NA'                    ) { return 'nc_tod'; }"
                        "else if (v=='LRT|NA'                    ) { return 'nc_tod'; }"
                        "else                                      { return 'other' ; }")

#symbology for enrollment classes
uv_aa = [  
          {"value":"mc"    , "label":"Metropolitan Center"         , "symbol":{"type":"esriSFS","color":[128,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"mc_tod", "label":"Metropolitan Center with TOD", "symbol":{"type":"esriSFS","color":[128,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"uc"    , "label":"Urban Center"                , "symbol":{"type":"esriSFS","color":[  0,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"uc_tod", "label":"Urban Center with TOD"       , "symbol":{"type":"esriSFS","color":[  0,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"cc"    , "label":"City Center"                 , "symbol":{"type":"esriSFS","color":[255,  0,  0,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"cc_tod", "label":"City Center with TOD"        , "symbol":{"type":"esriSFS","color":[255,  0,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"nc_tod", "label":"TOD Only"                    , "symbol":{"type":"esriSFS","color":[  0,255,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"other" , "label":"Other"                       , "symbol":{"type":"esriSFS","color":[128,128,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}
        ]

#define sdf layer
sdf_AA.spatial.plot(map_widget = map_areas,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_areas.layout.height='500px'
map_areas.legend=True

#map title
display(Markdown('<h2><center>Analysis Areas</center></h2>'))

#display map
map_areas

<h2><center>Analysis Areas</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Parcels

In [6]:
sdf_ParcelsAA = pd.DataFrame.spatial.from_featureclass(ParcelsAA)
sdf_ParcelsAA = sdf_ParcelsAA.rename(columns={"area_id": "AreaID", "class_id": "ClassID", "acres":"Acres"}, errors="raise")
sdf_ParcelsAA.groupby(['AreaID','ClassID'], as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "Acres":[np.sum]})

,AreaID,ClassID,parcel_id,job_spaces,resunits,Acres
,,,size,sum,sum,sum
0,0,2,89.0,0.000000,6.453172,43.947828
1,1,0,154.0,492.716357,3.413275,64.707475
2,1,8,852.0,3985.105851,203.013940,256.948341
3,2,8,77.0,2292.151510,0.995514,117.049989
4,5,8,328.0,1564.073690,792.140384,773.908744
...,...,...,...,...,...,...
251,220,6,27.0,0.000000,12.489516,2.735002
252,221,4,247.0,1313.711415,100.329383,67.878284
253,221,8,943.0,4326.394728,1593.984044,368.855866


In [7]:
sdf_ParcelsAA['county_id']

0         2.0
1         2.0
2         2.0
3         2.0
4         2.0
         ... 
126283    3.0
126284    3.0
126285    3.0
126286    3.0
126287    3.0
Name: county_id, Length: 126288, dtype: float64

# Land Use

In [8]:
display(sdf_ParcelsAA.columns)
sdf_ParcelsAA = sdf_ParcelsAA.rename(columns={"lu_type": "luTypeEx"})
display(sdf_ParcelsAA.columns)

Index(['FID', 'Shape_Leng', 'Shape_Area', 'fid_analys', 'AreaID', 'ClassID',
       'fid_parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'lu_type', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'parcel_a_1', 'piece_acre',
       'piece_port', 'Acres', 'SHAPE'],
      dtype='object')

Index(['FID', 'Shape_Leng', 'Shape_Area', 'fid_analys', 'AreaID', 'ClassID',
       'fid_parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'parcel_a_1', 'piece_acre',
       'piece_port', 'Acres', 'SHAPE'],
      dtype='object')

In [9]:
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & (sdf_ParcelsAA['bldgtype'] == 'Residential'), 'luTypeEx'] = "Residential"
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & (sdf_ParcelsAA['bldgtype'] == 'Commercial' ), 'luTypeEx'] = "Commercial"
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & ((sdf_ParcelsAA['bldgtype'] != 'Residential') & (sdf_ParcelsAA['bldgtype'] != 'Commercial')), 'luTypeEx'] = "Developable"

In [10]:
#create map centered on Salt Lake
map_parcels_undevelopable = gis.map('Salt Lake')

countyfilter = 2


#Filter by County
sdf_ParcelsAA_lu = sdf_ParcelsAA[sdf_ParcelsAA['county_id']==countyfilter]
#sdf_ParcelsAA_lu

#classify land use
arcade_expression_lu = ("var v = $feature.luTypeEx;"
                        "if      (v=='Residential'  ) { return 'res'  ; }"
                        "else if (v=='Commercial'   ) { return 'com'  ; }"
                        "else if (v=='Developable'  ) { return 'dev'  ; }"
                        "else if (v=='Undevelopable') { return 'undev'; }"
                        "else                           { return 'other'  ; }")

#symbology for enrollment classes
uv_lu = [{"value":"res"  , "label":"Residential"  , "symbol":{"type":"esriSFS","color":[255,255,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
         {"value":"com"  , "label":"Commercial"   , "symbol":{"type":"esriSFS","color":[255,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
         {"value":"dev"  , "label":"No Building"  , "symbol":{"type":"esriSFS","color":[  0,  0,255,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
         {"value":"undev", "label":"Undevelopable", "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsAA_lu.spatial.plot(map_widget = map_parcels_undevelopable,
                              renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                              unique_values=uv_lu,
                              arcade_expression=arcade_expression_lu,
                              default_symbol="" #don't include an 'other' category
                              )

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_undevelopable,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_undevelopable.layout.height='500px'
map_parcels_undevelopable.legend=True

#map title
#display(Markdown('<h2><center>Land Use - County ' + str(countyfilter) + '</center></h2>'))
display(Markdown('<h2><center>Existing Land Use</center></h2>'))

#display map
map_parcels_undevelopable

C:\Users\bhereth\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\pandas\core\series.py:4469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


<h2><center>Existing Land Use</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Building Age

In [11]:
sdf_ParcelsAA.columns

Index(['FID', 'Shape_Leng', 'Shape_Area', 'fid_analys', 'AreaID', 'ClassID',
       'fid_parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'parcel_a_1', 'piece_acre',
       'piece_port', 'Acres', 'SHAPE'],
      dtype='object')

In [12]:
#set building age with base year 2019
sdf_ParcelsAA['BldAge'] = 0
sdf_ParcelsAA.loc[((sdf_ParcelsAA['yearbuilt'] > 0)), 'BldAge'] = 2019 - sdf_ParcelsAA['yearbuilt']
display(sdf_ParcelsAA)
display(sdf_ParcelsAA.columns)

,FID,Shape_Leng,Shape_Area,fid_analys,AreaID,ClassID,fid_parcel,parcel_id,parcel_id_,basebldg,...,sfvalue_re,sfvalue_co,value_res,value_com,parcel_a_1,piece_acre,piece_port,Acres,SHAPE,BldAge
0,0,0.003171,6.168313e-07,3,5,8,178874,343987.0,343987.0,0.0,...,0.0,0.000000,0.0,0.000000,1.434973,1.435080,1.000074,1.435080,"{""rings"": [[[-111.91780621899994, 40.483768244...",0.0
1,1,0.001777,1.031191e-07,3,5,8,178875,343982.0,343982.0,0.0,...,0.0,0.000000,0.0,0.000000,0.239895,0.239912,1.000073,0.239912,"{""rings"": [[[-111.91668200699996, 40.483636867...",0.0
2,2,0.005323,5.916069e-07,3,5,8,182211,344598.0,344598.0,0.0,...,0.0,0.000000,0.0,0.000000,5.082228,1.376685,0.270882,1.376685,"{""rings"": [[[-111.92929745099997, 40.469735348...",0.0
3,3,0.001180,6.206041e-08,3,5,8,186793,344086.0,344086.0,1.0,...,0.0,52.966887,0.0,268244.148012,0.144383,0.144394,1.000074,0.144394,"{""rings"": [[[-111.91366418099994, 40.480443458...",16.0
4,4,0.001175,5.857432e-08,3,5,8,189951,344024.0,344024.0,1.0,...,0.0,70.171318,0.0,547379.407565,0.136263,0.136274,1.000079,0.136274,"{""rings"": [[[-111.91229344199996, 40.484995363...",17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126283,126283,0.000987,5.322657e-08,186,113,8,648134,549671.0,549671.0,0.0,...,0.0,0.000000,0.0,0.000000,0.171895,0.122341,0.711723,0.122341,"{""rings"": [[[-111.99983153599999, 41.298872987...",0.0
126284,126284,0.000987,5.322657e-08,186,113,8,665078,546641.0,546641.0,0.0,...,0.0,0.000000,0.0,0.000000,0.171895,0.122341,0.711723,0.122341,"{""rings"": [[[-111.99983153599999, 41.298872987...",0.0
126285,126285,0.000987,5.322657e-08,186,113,8,665079,545690.0,545690.0,0.0,...,0.0,0.000000,0.0,0.000000,0.171895,0.122341,0.711723,0.122341,"{""rings"": [[[-111.99983153599999, 41.298872987...",0.0
126286,126286,0.000987,5.322657e-08,186,113,8,685145,548151.0,548151.0,0.0,...,0.0,0.000000,0.0,0.000000,0.171895,0.122341,0.711723,0.122341,"{""rings"": [[[-111.99983153599999, 41.298872987...",0.0


Index(['FID', 'Shape_Leng', 'Shape_Area', 'fid_analys', 'AreaID', 'ClassID',
       'fid_parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'parcel_a_1', 'piece_acre',
       'piece_port', 'Acres', 'SHAPE', 'BldAge'],
      dtype='object')

In [13]:
#create map centered on Salt Lake
map_parcels_bldgage = gis.map('Salt Lake')


countyfilter = 2

#Filter by County
sdf_ParcelsAA_bldg = sdf_ParcelsAA[(sdf_ParcelsAA['county_id']==countyfilter) & (sdf_ParcelsAA['luTypeEx']!='Undevelopable')]
#sdf_ParcelsAA_lu

#expression to classify enrollment
arcade_expression_bldgage = ("var v = $feature.BldAge;"
                             "if      (v==0)  { return 'class1'; }"
                             "else if (v<40)  { return 'class2'; }"
                             "else if (v<80)  { return 'class3'; }"
                             "else if (v<120) { return 'class3'; }"
                             "else            { return 'class4'; }")

#symbology for enrollment classes
uv_public = [{"value":"class1", "label":"No Building"  , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
             {"value":"class2", "label":"Less than 40" , "symbol":{"type":"esriSFS","color":[143,111,146,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
             {"value":"class3", "label":"40 to 80"     , "symbol":{"type":"esriSFS","color":[155, 58, 75,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
             {"value":"class4", "label":"80 to 120"    , "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
             {"value":"class4", "label":"More than 120", "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsAA_bldg.spatial.plot(map_widget = map_parcels_bldgage,
                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                unique_values=uv_public,
                                arcade_expression=arcade_expression_bldgage,
                                default_symbol="" #don't include an 'other' category
                                )

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_bldgage,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_bldgage.layout.height='500px'
map_parcels_bldgage.legend=True

#map title
#display(Markdown('<h2><center>Building Age by Parcel - County ' + str(countyfilter) + '</center></h2>'))
display(Markdown('<h2><center>Building Age (Years) by Parcel</center></h2>'))

#display map
map_parcels_bldgage

C:\Users\bhereth\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\pandas\core\series.py:4469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


<h2><center>Building Age (Years) by Parcel</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Parcel Value

In [14]:
sdf_ParcelsAA.columns

Index(['FID', 'Shape_Leng', 'Shape_Area', 'fid_analys', 'AreaID', 'ClassID',
       'fid_parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'parcel_a_1', 'piece_acre',
       'piece_port', 'Acres', 'SHAPE', 'BldAge'],
      dtype='object')

In [15]:
#set building age with base year 2019
sdf_ParcelsAA['ValuePerAcre'] = 0
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Residential')), 'ValuePerAcre'] = sdf_ParcelsAA['value_res'] / sdf_ParcelsAA['Acres']
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Commercial' )), 'ValuePerAcre'] = sdf_ParcelsAA['value_com'] / sdf_ParcelsAA['Acres']

#classify existing acreage by existing land use

sdf_ParcelsAA['acres_res'] = 0.0
sdf_ParcelsAA['acres_com'] = 0.0

sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Residential')), 'acres_res'] = sdf_ParcelsAA['Acres']
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Commercial' )), 'acres_com'] = sdf_ParcelsAA['Acres']

display(sdf_ParcelsAA)

,FID,Shape_Leng,Shape_Area,fid_analys,AreaID,ClassID,fid_parcel,parcel_id,parcel_id_,basebldg,...,value_com,parcel_a_1,piece_acre,piece_port,Acres,SHAPE,BldAge,ValuePerAcre,acres_res,acres_com
0,0,0.003171,6.168313e-07,3,5,8,178874,343987.0,343987.0,0.0,...,0.000000,1.434973,1.435080,1.000074,1.435080,"{""rings"": [[[-111.91780621899994, 40.483768244...",0.0,0.000000e+00,0.0,0.000000
1,1,0.001777,1.031191e-07,3,5,8,178875,343982.0,343982.0,0.0,...,0.000000,0.239895,0.239912,1.000073,0.239912,"{""rings"": [[[-111.91668200699996, 40.483636867...",0.0,0.000000e+00,0.0,0.000000
2,2,0.005323,5.916069e-07,3,5,8,182211,344598.0,344598.0,0.0,...,0.000000,5.082228,1.376685,0.270882,1.376685,"{""rings"": [[[-111.92929745099997, 40.469735348...",0.0,0.000000e+00,0.0,0.000000
3,3,0.001180,6.206041e-08,3,5,8,186793,344086.0,344086.0,1.0,...,268244.148012,0.144383,0.144394,1.000074,0.144394,"{""rings"": [[[-111.91366418099994, 40.480443458...",16.0,1.857726e+06,0.0,0.144394
4,4,0.001175,5.857432e-08,3,5,8,189951,344024.0,344024.0,1.0,...,547379.407565,0.136263,0.136274,1.000079,0.136274,"{""rings"": [[[-111.91229344199996, 40.484995363...",17.0,4.016770e+06,0.0,0.136274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126283,126283,0.000987,5.322657e-08,186,113,8,648134,549671.0,549671.0,0.0,...,0.000000,0.171895,0.122341,0.711723,0.122341,"{""rings"": [[[-111.99983153599999, 41.298872987...",0.0,0.000000e+00,0.0,0.000000
126284,126284,0.000987,5.322657e-08,186,113,8,665078,546641.0,546641.0,0.0,...,0.000000,0.171895,0.122341,0.711723,0.122341,"{""rings"": [[[-111.99983153599999, 41.298872987...",0.0,0.000000e+00,0.0,0.000000
126285,126285,0.000987,5.322657e-08,186,113,8,665079,545690.0,545690.0,0.0,...,0.000000,0.171895,0.122341,0.711723,0.122341,"{""rings"": [[[-111.99983153599999, 41.298872987...",0.0,0.000000e+00,0.0,0.000000
126286,126286,0.000987,5.322657e-08,186,113,8,685145,548151.0,548151.0,0.0,...,0.000000,0.171895,0.122341,0.711723,0.122341,"{""rings"": [[[-111.99983153599999, 41.298872987...",0.0,0.000000e+00,0.0,0.000000


In [16]:
#create map centered on Salt Lake
map_parcels_areas_value = gis.map('Salt Lake')



countyfilter = 2

#Filter by County
sdf_ParcelsAA_value = sdf_ParcelsAA[(sdf_ParcelsAA['county_id']==countyfilter) & (sdf_ParcelsAA['luTypeEx'] == 'Residential')]

#(sdf_ParcelsAA['luTypeEx']!='Undevelopable') & 
#sdf_ParcelsAA_lu

#expression to classify employment growth
#don't use value if 0
arcade_expression_value = ("var v = $feature.ValuePerAcre;"
                           "if      (v>0       && v<=1000000) { return 'class4'; }"
                           "else if (v>1000000 && v<=3000000) { return 'class3'; }"
                           "else if (v>3000000 && v<=5000000) { return 'class2'; }"
                           "else if (v>5000000              ) { return 'class1'; }")

#symbology for enrollment classes
uv_value = [{"value":"class1", "label":"More than $5m/acre" , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class2", "label":"$5m to $3m/acre"   , "symbol":{"type":"esriSFS","color":[143,111,146,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class3", "label":"$3m to $1m/acre"   , "symbol":{"type":"esriSFS","color":[155, 58, 75,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class4", "label":"Less than $1m/acre", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsAA_value.spatial.plot(map_widget = map_parcels_areas_value,
                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                unique_values=uv_value,
                                arcade_expression=arcade_expression_value,
                                default_symbol="" #don't include an 'other' category
                                )

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_areas_value,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_areas_value.layout.height='500px'
map_parcels_areas_value.legend=True

#map title
display(Markdown('<h2><center>Value per Acre - Residential</center></h2>'))

#display map
map_parcels_areas_value

C:\Users\bhereth\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\pandas\core\series.py:4469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


<h2><center>Value per Acre - Residential</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

In [17]:
#create map centered on Salt Lake
map_parcels_areas_value = gis.map('Salt Lake')



countyfilter = 2

#Filter by County
sdf_ParcelsAA_value = sdf_ParcelsAA[(sdf_ParcelsAA['county_id']==countyfilter) & (sdf_ParcelsAA['luTypeEx'] == 'Commercial'))]
#sdf_ParcelsAA_lu

#expression to classify employment growth
#don't use value if 0
arcade_expression_value = ("var v = $feature.ValuePerAcre;"
                           "if      (v>0       && v<=1000000) { return 'class4'; }"
                           "else if (v>1000000 && v<=3000000) { return 'class3'; }"
                           "else if (v>3000000 && v<=5000000) { return 'class2'; }"
                           "else if (v>5000000              ) { return 'class1'; }")

#symbology for enrollment classes
uv_value = [{"value":"class1", "label":"More than 5m/acre" , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class2", "label":"$3m to $5m/acre"   , "symbol":{"type":"esriSFS","color":[143,111,146,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class3", "label":"$1m to $3m/acre"   , "symbol":{"type":"esriSFS","color":[155, 58, 75,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class4", "label":"Less than $1m/acre", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsAA_value.spatial.plot(map_widget = map_parcels_areas_value,
                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                unique_values=uv_value,
                                arcade_expression=arcade_expression_value,
                                default_symbol="" #don't include an 'other' category
                                )

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_areas_value,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_areas_value.layout.height='500px'
map_parcels_areas_value.legend=True

#map title
display(Markdown('<h2><center>Value per Acre - Commercial</center></h2>'))

#display map
map_parcels_areas_value

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 22)

# Development Categories

In [19]:
df_ClassParam = pd.read_csv(ClassParameters)
display(df_ClassParam)

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,ClassOrder
0,1,CRT|Metropolitan Center,0.40,0.60,350,8,1000,500,0.05,650000,400000,AND,60,120,30,120,1
1,5,LRT|Metropolitan Center,0.40,0.60,350,8,1000,500,0.05,650000,400000,AND,60,120,30,120,2
2,9,NONTOD|Metropolitan Center,0.45,0.55,170,4,1000,500,0.05,650000,400000,AND,60,120,30,120,3
3,3,CRT|Urban Center,0.50,0.50,130,3,1000,500,0.10,500000,300000,AND,70,300,30,300,4
4,7,LRT|Urban Center,0.50,0.50,130,3,1000,500,0.10,500000,300000,AND,70,300,30,300,5
5,10,NONTOD|Urban Center,0.55,0.45,70,2,1200,500,0.10,500000,300000,AND,70,300,30,300,6
6,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7
7,4,LRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,8
8,8,NONTOD|City Center,0.65,0.35,35,1,1200,600,0.10,500000,300000,AND,80,300,40,300,9
9,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10


In [21]:
#ReDevelop, NewDevelop, NoDevelop

sdf_ParcelsAAwParam = pd.DataFrame.merge(sdf_ParcelsAA,df_ClassParam,on='ClassID',how='left')
#display(sdf_ParcelsAAwParam)


#sdf_ParcelsAAwParam = sdf_ParcelsAAwParam.rename(columns={"lu_type_ex": "luTypeEx", "value_per_acre": "ValuePerAcre", "bld_age":"BldAge", "dev_category":"DevCategory"}, errors="raise")

#undeveloped
sdf_ParcelsDevCat = sdf_ParcelsAAwParam.copy()
sdf_ParcelsDevCat['DevCategory'] = 'Remain'
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['luTypeEx'] == 'Developable'), 'DevCategory'] = "Developable"

#check if redevelopable - step 1: building age
sdf_ParcelsDevCat['DevCategoryAge'] = ''
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['BldAge'] >= sdf_ParcelsDevCat['RedevBldgAgeRes_Low']) & (sdf_ParcelsDevCat['BldAge'] <= sdf_ParcelsDevCat['RedevBldgAgeRes_High']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Residential'), 'DevCategoryAge'] = "Redevelopable"
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['BldAge'] >= sdf_ParcelsDevCat['RedevBldgAgeCom_Low']) & (sdf_ParcelsDevCat['BldAge'] <= sdf_ParcelsDevCat['RedevBldgAgeCom_High']) & (sdf_ParcelsDevCat['luTypeEx'] != 'Residential'), 'DevCategoryAge'] = "Redevelopable"

#check if redevelopable - step 2: value
sdf_ParcelsDevCat['DevCategoryValue'] = ''
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['ValuePerAcre'] > 0) & (sdf_ParcelsDevCat['ValuePerAcre'] <= sdf_ParcelsDevCat['RedevValuePerAcreRes']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Residential'), 'DevCategoryValue'] = "Redevelopable"
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['ValuePerAcre'] > 0) & (sdf_ParcelsDevCat['ValuePerAcre'] <= sdf_ParcelsDevCat['RedevValuePerAcreCom']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Commercial' ), 'DevCategoryValue'] = "Redevelopable"

sdf_ParcelsDevCat.loc[ (sdf_ParcelsDevCat['RedevAndOr'] =='AND') &  (sdf_ParcelsDevCat['DevCategoryAge'] == 'Redevelopable') & (sdf_ParcelsDevCat['DevCategoryValue'] == 'Redevelopable')  , 'DevCategory'] = "Redevelopable"
sdf_ParcelsDevCat.loc[((sdf_ParcelsDevCat['RedevAndOr'] =='OR' ) & ((sdf_ParcelsDevCat['DevCategoryAge'] == 'Redevelopable') | (sdf_ParcelsDevCat['DevCategoryValue'] == 'Redevelopable'))), 'DevCategory'] = "Redevelopable"

#undevelopable acres
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['luTypeEx'] == 'Undevelopable'), 'DevCategory'] = "Undevelopable"

#export to GeoDB
display("exporting to gdb...")
deleteIfExists(ParcelsDevCategory)
sdf_ParcelsDevCat.spatial.to_featureclass(location=ParcelsDevCategory)

#display(sdf_ParcelsDevCat[['yearbuilt','luTypeEx','ValuePerAcre','BldAge','DevCategory']])

sdf_ParcelsDevCat.columns

'exporting to gdb...'

Index(['fid', 'shape_leng', 'shape_area', 'fid_analys', 'area_id', 'class_id',
       'fid_parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'lu_type_ex', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'parcel_a_1', 'piece_acre',
       'piece_port', 'acres', 'SHAPE', 'bld_age', 'value_per_acre',
       'acres_res', 'acres_com', 'class_description', 'sf_split_res',
       'sf_split_com', 'capacity_res_dua', 'capacity_com_far', 's_fper_hh',
       's_fper_emp', 'percent_open_space', 'redev_value_per_acre_res',
       'redev_value_per_acre_com', 'redev_and_or', 'redev_bldg_age_res_low',
       'redev_bldg_age_res_high', 'redev_bldg_age_com_low',
       'redev_bldg_age_com_high', 'class_order', 'dev_category',
       'dev_category_age', 'dev_category_value'],
      dtype='object')

In [22]:
sdf_ParcelsWebApp = sdf_ParcelsDevCat[['county_id','area_id','class_id','parcel_id','lu_type_ex','bld_age','value_per_acre','dev_category','piece_acre','job_spaces','resunits','SHAPE']]
#sdf_ParcelsWebApp

#if processing geodatabase doesn't exist, create it
print("Checking if " + WebAppGDB + " exists...")
if not arcpy.Exists(os.path.join(dir_results, WebAppGDB)):
    print("Creating " + WebAppGDB + "...")
    arcpy.management.CreateFileGDB(dir_results, WebAppGDB)
else:
    print(WebAppGDB + " exists...")

#export to GeoDB
display("exporting to gdb...")
deleteIfExists(ParcelsWebApp)
sdf_ParcelsWebApp.spatial.to_featureclass(location=ParcelsWebApp)
display("repairing geometry...")
arcpy.RepairGeometry_management(ParcelsWebApp)

Checking if centerstod_webapp.gdb exists...
centerstod_webapp.gdb exists...


'exporting to gdb...'

C:\Users\bhereth\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


'repairing geometry...'

<Result 'e:\\GitHub\\Centers-Capacity-Tool-byCity\\forWebApp\\centerstod_webapp.gdb\\AAParcels'>

In [23]:
#dissolve on DevCategory
deleteIfExists(DevCatWebApp)
arcpy.management.Dissolve(ParcelsWebApp, DevCatWebApp, dissolve_field=['area_id','class_id','dev_category'], statistics_fields=[['piece_acre','SUM'],['job_spaces','SUM'],['resunits','SUM']])

#repair the layer
arcpy.RepairGeometry_management(DevCatWebApp)

<Result 'e:\\GitHub\\Centers-Capacity-Tool-byCity\\forWebApp\\centerstod_webapp.gdb\\AADevCategories'>

In [24]:
display(df_ClassParam)
df_ClassParam.dtypes

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,ClassOrder
0,1,CRT|Metropolitan Center,0.40,0.60,350,8,1000,500,0.05,650000,400000,AND,60,120,30,120,1
1,5,LRT|Metropolitan Center,0.40,0.60,350,8,1000,500,0.05,650000,400000,AND,60,120,30,120,2
2,9,NONTOD|Metropolitan Center,0.45,0.55,170,4,1000,500,0.05,650000,400000,AND,60,120,30,120,3
3,3,CRT|Urban Center,0.50,0.50,130,3,1000,500,0.10,500000,300000,AND,70,300,30,300,4
4,7,LRT|Urban Center,0.50,0.50,130,3,1000,500,0.10,500000,300000,AND,70,300,30,300,5
5,10,NONTOD|Urban Center,0.55,0.45,70,2,1200,500,0.10,500000,300000,AND,70,300,30,300,6
6,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7
7,4,LRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,8
8,8,NONTOD|City Center,0.65,0.35,35,1,1200,600,0.10,500000,300000,AND,80,300,40,300,9
9,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10


ClassID                   int64
ClassDescription         object
SFSplitRes              float64
SFSplitCom              float64
CapacityRes_DUA           int64
CapacityCom_FAR           int64
SFperHH                   int64
SFperEmp                  int64
PercentOpenSpace        float64
RedevValuePerAcreRes      int64
RedevValuePerAcreCom      int64
RedevAndOr               object
RedevBldgAgeRes_Low       int64
RedevBldgAgeRes_High      int64
RedevBldgAgeCom_Low       int64
RedevBldgAgeCom_High      int64
ClassOrder                int64
dtype: object

In [25]:
countyfilter = 2

#Filter by County
sdf_ParcelsDevCat_filter = sdf_ParcelsDevCat.copy()
sdf_ParcelsDevCat_filter = sdf_ParcelsDevCat_filter[sdf_ParcelsDevCat_filter['county_id']==countyfilter]

#create map centered on Salt Lake
map_parcels_dev = gis.map('Salt Lake')

#expression to classify enrollment
arcade_expression_dev = ("var v = $feature.DevCategory;"
                             "if      (v=='Undevelopable') { return 'class2'; }"
                             "else if (v=='Remain'       ) { return 'class2'; }"
                             "else if (v=='Redevelopable') { return 'class3'; }"
                             "else if (v=='Developable'  ) { return 'class4'; }"
                        )

#symbology for enrollment classes
uv_dev = [{"value":"class4", "label":"Developable"  , "symbol":{"type":"esriSFS","color":[  0,  0,255,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"class3", "label":"Redevelopable", "symbol":{"type":"esriSFS","color":[128,  0,128,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"class2", "label":"To Remain"    , "symbol":{"type":"esriSFS","color":[180,180,180,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsDevCat_filter.spatial.plot(map_widget = map_parcels_dev,
                                      renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                      unique_values=uv_dev,
                                      arcade_expression=arcade_expression_dev,
                                      default_symbol="" #don't include an 'other' category
                                      )
#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_dev,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_dev.layout.height='500px'
map_parcels_dev.legend=True

#map title
display(Markdown('<h2><center>Development Category - County ' + str(countyfilter) + '</center></h2>'))

#display map
map_parcels_dev

<h2><center>Development Category - County 2</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

In [26]:
#aggregate parcels into analysis areas / classes
df_AADevCat = sdf_ParcelsDevCat.groupby(['area_id','class_id','county_id','dev_category'],as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "acres":[np.sum], "sf_res":[np.sum], "sf_com":[np.sum], "acres_res":[np.sum], "acres_com":[np.sum]})
df_AADevCat.columns = df_AADevCat.columns.droplevel(1)
display(df_AADevCat)


df_AADevCat.to_csv(os.path.join(dir_results, r'AnalysisAreas_byDevCategory.csv'),index=False)


,area_id,class_id,county_id,dev_category,parcel_id,job_spaces,resunits,acres,sf_res,sf_com,acres_res,acres_com
0,0,2,4.0,Developable,46.0,0.000000,0.000000,30.777796,0.000000,0.000000,0.000000,0.000000
1,0,2,4.0,Remain,21.0,0.000000,6.391683,6.426645,25060.942589,0.000000,6.426645,0.000000
2,0,2,4.0,Undevelopable,22.0,0.000000,0.061490,6.743387,180.410686,0.000000,0.000000,0.000000
3,1,0,4.0,Developable,98.0,0.000000,0.000000,21.879749,0.000000,0.000000,0.000000,0.000000
4,1,0,4.0,Remain,33.0,492.716357,3.413275,22.210932,7539.391832,184465.743750,0.894349,21.316583
...,...,...,...,...,...,...,...,...,...,...,...,...
918,222,8,1.0,Undevelopable,11.0,0.000000,0.000000,6.111900,0.000000,0.000000,0.000000,0.000000
919,223,2,1.0,Developable,58.0,0.000000,0.000000,33.147583,0.000000,0.000000,0.000000,0.000000
920,223,2,1.0,Redevelopable,9.0,26.051029,3.999483,5.457562,6689.134478,24977.482447,1.389284,4.068278
921,223,2,1.0,Remain,334.0,388.671084,296.733379,72.605319,371485.457837,206880.106991,61.060865,11.544454
